In [1]:
import sys

sys.path.append("/Users/Mike/desktop/UCSB/CS/CS111/cs111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import cs111
from tabulate import tabulate

%matplotlib ipympl

np.set_printoptions(precision = 4)

## Homework 4 Problem 1

#### Constructing the Force Matrix $F$

In [128]:
# equations are numbered from 0
# variables in the book are numbered from 1
alpha = 1/math.sqrt(2)
F = np.zeros((13,13)) # this will be our forces matrix
b = np.zeros(13) # this will be our member forces vector
b[[1,7,9]]=[10,15,20] # all zeros except equations 1, 7, and 9

# we proceed to make the equations as 2D arraus
eq0  = np.array([[1,5]    , [1,-1]]) # f2-f6=
eq1  = np.array([[2]      , [1]]) # f3=
eq2  = np.array([[0,3,4]  , [alpha,-1,-alpha]]) # alpha*f1-f4-alpha*f5=
eq3  = np.array([[0,2,4]  , [alpha,1,alpha]]) # alpha*f1+f3+alpha*f5=
eq4  = np.array([[3,7]    , [1,-1]]) # same logic for the rest
eq5  = np.array([[6]      , [1]])
eq6  = np.array([[4,5,8,9], [alpha,1,-alpha,-1]])
eq7  = np.array([[4,6,8]  , [alpha,1,alpha]])
eq8  = np.array([[9,12]   , [1,-1]])
eq9  = np.array([[10]     , [1]])
eq10 = np.array([[7,8,11] , [1,alpha,-alpha]])
eq11 = np.array([[8,10,11], [alpha,1,alpha]])
eq12 = np.array([[12,11]  , [1,alpha]])

# make an array of our equations so we can loop through them
eqArr = [eq0,eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12]
for i in range(F.shape[0]):
    F[i,eqArr[i][0][:].astype(int)] = eqArr[i][1][:]
#print(F) # to check the equation matrix looks correct

In [96]:
f_1 = npla.solve(F,b) # regular solve
f_2, relres = cs111.LUsolve(F,b) # check with cs111.LUsolve()
print("npla.solve(F,b) == cs111.LUsolve(F,b)?", (np.round(f_1,4)==np.round(f_2,4)).all()) # compare solutions
print("relative residual norm: ", relres, "\n")
print("Given the 'loads' vector b: \n", b)
print("Our vector of 'member forces' is \n", f_1)

npla.solve(F,b) == cs111.LUsolve(F,b)? True
relative residual norm:  2.0064672503295845e-16 

Given the 'loads' vector b: 
 [ 0. 10.  0.  0.  0.  0.  0. 15.  0. 20.  0.  0.  0.]
Our vector of 'member forces' is 
 [-28.2843  20.      10.     -30.      14.1421  20.       0.     -30.
   7.0711  25.      20.     -35.3553  25.    ]


## Homework 4 Problem 2.1

In [99]:
# first we find the A to satisfy conditions
x = np.zeros(2)
foundA = np.zeros((2,2))
found = False
while(not found):
    foundA = np.zeros((2,2))
    A = np.random.rand(2,2)
    Asymm = (A+A.T)/2
    n, m = Asymm.shape
    if(n == m and npla.matrix_rank(Asymm) == n and (Asymm[:,:] > 0).all()):
        print("Found A under given conditions: \n", Asymm)   
        foundA = Asymm
        # then for this A we check 100 vectors to see if find one
        for i in range(100):
            temp = np.random.randn(2)
            if(temp.T@foundA@temp < 0):
                print("Found an x to satisfy x^T@A@x < 0: \n", temp)   
                print("x^T@A@x = ", temp.T@foundA@temp)
                x = temp
                found = True
                break
        if(not found):
            print("Did not find such an x for this matrix A \n \n")

Found A under given conditions: 
 [[0.8169 0.2842]
 [0.2842 0.2879]]
Did not find such an x for this matrix A 
 

Found A under given conditions: 
 [[0.6292 0.6543]
 [0.6543 0.2362]]
Found an x to satisfy x^T@A@x < 0: 
 [ 0.142  -0.3759]
x^T@A@x =  -0.0238076816278393


## Homework 4 Problem 2.2

We have $$A = B^TB,$$
where $B$ is a $m$-by-$n$ matrix with rank $n.$ We know $A$ is SPD if $$\forall x\neq 0, \text{} x^TAx > 0$$
We can rewrite $$x^TAx = x^TB^TBx=(Bx)^TBx=||Bx||^2$$
We know the rank of matrix $\text{ }B\text{ }$ is $\text{ }n\text{ }$, so its columns are linearly independent. We also know that $\text{ }x\neq0$. Thus, $$||Bx||^2 > 0 \Rightarrow x^TAx > 0 \Rightarrow \text{ A is SPD}$$

## Homework 4 Problem 3

Start with an n-by-n matrix $A=R^TR$:
$$\begin{pmatrix}a_{1,1} & a_{1,2} & \cdots & a_{1,n}\\
a_{2,1} & a_{2,2} & \cdots & a_{2,n}\\
\cdot & \cdot & &\\
\cdot & & \cdot &\\
a_{n-1,1} & a_{n-1,2} & \cdots & a_{n-1,n}\\
a_{n,1} & a_{n,2} & \cdots & a_{n,n}\end{pmatrix} = \begin{pmatrix}R_{1,1} & \cdots & 0 & 0\\
R_{2,1} & \cdots & 0 & 0\\
\cdot & \cdot & &\\
\cdot & & \cdot &\\
R_{n-1,1} & R_{n-1,2} & \cdots & 0\\
R_{n,1} & R_{n,2} & \cdots & R_{n,n}\end{pmatrix} \begin{pmatrix}R_{1,1} & R_{2,1} & \cdots & R_{n,1}\\
0 & L_{2,2} & \cdots & R_{n,2}\\
\cdot & \cdot & &\\
\cdot & & \cdot & \\
0 & \cdots & R_{n-1,n-1} & R_{n,n-1}\\
0 & \cdots & 0 & R_{n,n}\end{pmatrix}$$

$$A \text{ is symmetric } \Rightarrow A = A^T$$

So, we can solely look at the upper triangle of $A$, and calculate the matrix by only looking at half the terms:
$$
\begin{pmatrix}a_{1,1} & a_{1,2} & \cdots & a_{1,n}\\
0 & a_{2,2} & \cdots & a_{2,n}\\
\cdot & \cdot & &\\
\cdot & & \cdot &\\
0 & 0 & \cdots & a_{n-1,n}\\
0 & 0 & \cdots & a_{n,n}\end{pmatrix} = \begin{pmatrix}R_{1,1} & \cdots & 0 & 0\\
R_{2,1} & \cdots & 0 & 0\\
\cdot & \cdot & &\\
\cdot & & \cdot &\\
R_{n-1,1} & R_{n-1,2} & \cdots & 0\\
R_{n,1} & R_{n,2} & \cdots & R_{n,n}\end{pmatrix} \begin{pmatrix}R_{1,1} & R_{2,1} & \cdots & R_{n,1}\\
0 & L_{2,2} & \cdots & R_{n,2}\\
\cdot & \cdot & &\\
\cdot & & \cdot & \\
0 & \cdots & R_{n-1,n-1} & R_{n,n-1}\\
0 & \cdots & 0 & R_{n,n}\end{pmatrix}
$$

After multiplying, compare the individual elements of the result to the individual elements of our symmetric matrix $A$. Setting the diagonals of each matrix to each other warrants the following relation:
$
\forall i=j, \text{ } L_{i,j}=\sqrt{a_{i,j} \text{ }- \sum_{k=1}^{j-1}R_{i,k}^2}
$

Furthermore, the nondiagonal terms can be related by the following 
relation:
$
\forall i\neq j, \text{ } R_{i,j}=\frac{1}{R_{i,j}}(a_{i,j} - \sum_{k=1}^{j-1}R_{i,k}R_{j,k})
$

Manually writing out the equations for the first couple terms of matrix R, we notice a pattern

\begin{align*} r_{00}&=\sqrt{a_{00}} & r_{11}&=\sqrt{a_{11}-r_{01}r_{01}}   & r_{22}&=\sqrt{a_{22}-r_{02}r_{02}-r_{12}r_{12}}    & r_{33}&=\sqrt{a_{33}-r_{03}r_{03}-r_{13}r_{13}-r_{23}r_{23}} & \cdots \\ r_{01}&=a_{01}/r_{00} & r_{12}&=(a_{12}-r_{01}r_{02})/r_{11} & r_{23}&=(a_{23}-r_{02}r_{03}-r_{12}r_{13})/r_{22}  & ~       &~                                                     & \cdots \\ r_{02}&=a_{02}/r_{00} & r_{13}&=(a_{13}-r_{01}r_{03})/r_{11} & ~     &~                                           & ~       &~                                                     & \cdots \\ r_{03}&=a_{03}/r_{00} & ~     &~                             & ~     &~                                           & ~       &~                                                     & \cdots \\ \end{align*}

From this pattern, comes the Cholesky Algorithm for factoring SPD matrices:

In [100]:
# Cholesky Decomposition Algorithm of an SPD matrix A
# Pre-condition: A is SPD
# Post-condition: Returns an upper triangular
# matrix R such that A=R.T@R
def Cfactor(A):
    n = A.shape[0]
    R = np.triu(A)
    for i in range(n):
        R[i, i] **= 0.5
        R[i, i+1:] /= R[i, i]
        for j in range(i + 1, n):
            R[j, j:] -= R[i, j] * R[i, j:]
    return R

In [101]:
# Testing the Cholesky Decomposition Algorithm
# for correctness against cs111.LUsolve()
# test and inspect one random example
B = np.random.randn(10,10)
A = B.T @ B
b = np.random.randn(10)

LUsolveSolution, relres = cs111.LUsolve(A,b)

R = Cfactor(A)
y = cs111.Lsolve(R.T, b)
CholeskySolution = cs111.Usolve(R, y)

print("A = R.T@R? ", (np.round(R.T@R - A, 10) == 0).all(), "\n")
print("LUsolve Solution: \n", LUsolveSolution, "\n")
print("Cholesky Solution: \n", CholeskySolution, "\n")
print("Solutions equal? ", (np.round(LUsolveSolution-CholeskySolution, 6) == 0).all())

# check the correctness a couple more times
flag = True
for i in range(10000):
    B = np.random.randn(10,10)
    A = B.T @ B
    b = np.random.randn(10)
    LUsolveSolution, relres = cs111.LUsolve(A,b)
    R = Cfactor(A)
    CholeskySolution = cs111.Usolve(R, cs111.Lsolve(R.T, b))
    if (not (np.round(R.T@R - A, 6) == 0).all()):
        flag = False
print("Solution is correct" if flag else "Solution is wrong")

A = R.T@R?  True 

LUsolve Solution: 
 [ 0.2638 -1.8855 -0.7748 -1.3386 -0.2618 -0.1568  0.2993 -1.4988  0.4453
  0.5152] 

Cholesky Solution: 
 [ 0.2638 -1.8855 -0.7748 -1.3386 -0.2618 -0.1568  0.2993 -1.4988  0.4453
  0.5152] 

Solutions equal?  True
Solution is correct


In [92]:
# Timing Cfactor() against cs111.LUfactorNoPiv()
# plotting the ratio of the two runtimes against n
ratioResults = np.zeros(200)

for n in range(1, 1000, 5):
    B = np.random.randn(n,n)
    A = B.T @ B
    luTime = %timeit -n3 -r1 -q -p3 -o cs111.LUfactorNoPiv(A)
    cTime = %timeit -n3 -r1 -q -p3 -o Cfactor(A)
    ratioResults[int(n/5)] = cTime.average/luTime.average

In [93]:
# check some entries in the ratioResults,
ratioResults[150:]
# then plot it against size of matrix, n
plt.plot(range(200), ratioResults)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Homework 4 Problem 4

In [102]:
def findThirtySecK(func, begin, k, end, cs111LUsolve = False):
    A = cs111.make_A_3D(k)
    if (cs111LUsolve):
        A = A.toarray()
    b = A @ np.ones(A.shape[0])
    t = %timeit -n1 -r1 -q -p3 -o func(A,b)
    if (end - begin <= 1 or (t.average > 28 and t.average < 30)):
        print("The largest value of k for which this solver")
        print("solves Ax=b within 30 seconds is: ", k, "\n")
        print("Solved in: ", t.average)
    elif (t.average < 28):
        begin = k
        k = round((begin + end) / 2)
        print("k = ", begin, " too quick, trying k = ", k)
        findThirtySecK(func, begin, k, end, cs111LUsolve)
    else:
        end = k
        k = round((begin + end) / 2)
        print("k = ", end, " too slow, trying k = ", k)
        findThirtySecK(func, begin, k, end, cs111LUsolve)

In [8]:
arr = [cs111.CGsolve, cs111.Jsolve, scipy.sparse.linalg.cg, scipy.sparse.linalg.spsolve, cs111.LUsolve]
for item in arr:
    flag = False
    begin, k, end = 1, 100, 200
    funcName = item.__name__
    if (funcName == "spsolve"):
        begin, k, end = 1, 50, 100 # smaller range for the slower spsolve
    if (funcName == "LUsolve"):
        begin, k, end = 1, 15, 30 # an even smaller range for the slowest LUsolve
        flag = True
    print("Testing ", funcName, ":\n")
    findThirtySecK(item, begin, k, end, flag)
    print("\n\n")

Testing  CGsolve :

k =  100  too quick, trying k =  150
k =  150  too slow, trying k =  125
k =  125  too quick, trying k =  138
k =  138  too quick, trying k =  144
The largest value of k for which this solver
solves Ax=b within 30 seconds is:  144 

Solved in:  29.340680711999994



Testing  Jsolve :

k =  100  too quick, trying k =  150
k =  150  too slow, trying k =  125
k =  125  too slow, trying k =  112
k =  112  too slow, trying k =  106
The largest value of k for which this solver
solves Ax=b within 30 seconds is:  106 

Solved in:  29.13135038900009



Testing  cg :

k =  100  too quick, trying k =  150
k =  150  too quick, trying k =  175
k =  175  too slow, trying k =  162
k =  162  too quick, trying k =  168
k =  168  too quick, trying k =  172
k =  172  too quick, trying k =  174
The largest value of k for which this solver
solves Ax=b within 30 seconds is:  174 

Solved in:  28.71321223699988



Testing  spsolve :

k =  50  too slow, trying k =  26
k =  26  too quick, t

In [121]:
# testing speed and accuracy
# choose k = 15, tolerance = 10^-16, compare solvers
arr = [cs111.CGsolve, cs111.Jsolve, scipy.sparse.linalg.cg, scipy.sparse.linalg.spsolve, cs111.LUsolve]
names = arr
names = [item.__name__ for item in arr]
data = np.zeros((3,len(arr)))
k = 15

for i in range(len(arr)):
    name = arr[i].__name__
    A = cs111.make_A_3D(k)
    if (name == "LUsolve"):
        A = A.toarray()
    x_exact = np.ones(A.shape[0])
    b = A @ x_exact

    if (name == "CGsolve" or name == "Jsolve" or name == "cg"):
        t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b,tol=1e-10)
        data[0, i] = t.average
        x = arr[i](A,b,tol=1e-10)[0]
        data[1, i] = npla.norm(b-A@x)/npla.norm(b)
        data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
    else:
        t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b)
        data[0, i] = t.average
        if (name == "LUsolve"):
            x, relres = arr[i](A,b)
        else:
            x = arr[i](A,b)
            relres = npla.norm(b-A@x)/npla.norm(b)
        data[1, i] = relres
        data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)

In [122]:
print(tabulate(data, headers=names, tablefmt="grid"))

+-------------+-------------+-------------+-------------+--------------+
|     CGsolve |      Jsolve |          cg |     spsolve |      LUsolve |
+=============+=============+=============+=============+==============+
| 0.00545795  | 0.0765003   | 0.00299479  | 0.0416165   | 33.5182      |
+-------------+-------------+-------------+-------------+--------------+
| 3.92664e-11 | 4.835e-10   | 3.92664e-11 | 2.48315e-15 |  6.79081e-15 |
+-------------+-------------+-------------+-------------+--------------+
| 6.86191e-12 | 2.98519e-09 | 6.8619e-12  | 4.38191e-16 |  2.17806e-15 |
+-------------+-------------+-------------+-------------+--------------+


In [124]:
arr = [cs111.CGsolve, cs111.Jsolve, scipy.sparse.linalg.cg, scipy.sparse.linalg.spsolve, cs111.LUsolve]
names = arr
names = [item.__name__ for item in arr]
data = np.zeros((3,len(arr)))
for k in range(1,20,4):
    for i in range(len(arr)):
        name = arr[i].__name__
        A = cs111.make_A_3D(k)
        if (name == "LUsolve"):
            A = A.toarray()
        x_exact = np.ones(A.shape[0])
        b = A @ x_exact

        if (name == "CGsolve" or name == "Jsolve" or name == "cg"):
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b,tol=1e-10)
            data[0, i] = t.average
            x = arr[i](A,b,tol=1e-10)[0]
            data[1, i] = npla.norm(b-A@x)/npla.norm(b)
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
        else:
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b)
            data[0, i] = t.average
            if (name == "LUsolve"):
                x, relres = arr[i](A,b)
            else:
                x = arr[i](A,b)
                relres = npla.norm(b-A@x)/npla.norm(b)
            data[1, i] = relres
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
    print("k = ", k, "\n", tabulate(data, headers=names, tablefmt="grid"), "\n\n")

k =  1 
 +-------------+-------------+-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |     spsolve |     LUsolve |
+=============+=============+=============+=============+=============+
| 0.000176377 | 0.000324242 | 0.000222975 | 0.000496594 | 0.000240592 |
+-------------+-------------+-------------+-------------+-------------+
| 0           | 0           | 0           | 0           | 0           |
+-------------+-------------+-------------+-------------+-------------+
| 0           | 0           | 0           | 0           | 0           |
+-------------+-------------+-------------+-------------+-------------+ 


k =  5 
 +-------------+-------------+-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |     spsolve |     LUsolve |
+=============+=============+=============+=============+=============+
| 0.000384    | 0.00555654  | 0.000632041 | 0.000409092 | 0.0369408   |
+-------------+-------------+-------------+

In [125]:
# Now we remove LUsolve from the array of functions we want to test
# this way we can increase k more and really compare speed and precision
# of all the other solvers
arr = [cs111.CGsolve, cs111.Jsolve, scipy.sparse.linalg.cg, scipy.sparse.linalg.spsolve]
names = arr
names = [item.__name__ for item in arr]
data = np.zeros((3,len(arr)))
for k in range(1,42,10):
    for i in range(len(arr)):
        name = arr[i].__name__
        A = cs111.make_A_3D(k)
        if (name == "LUsolve"):
            A = A.toarray()
        x_exact = np.ones(A.shape[0])
        b = A @ x_exact

        if (name == "CGsolve" or name == "Jsolve" or name == "cg"):
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b,tol=1e-10)
            data[0, i] = t.average
            x = arr[i](A,b,tol=1e-10)[0]
            data[1, i] = npla.norm(b-A@x)/npla.norm(b)
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
        else:
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b)
            data[0, i] = t.average
            if (name == "LUsolve"):
                x, relres = arr[i](A,b)
            else:
                x = arr[i](A,b)
                relres = npla.norm(b-A@x)/npla.norm(b)
            data[1, i] = relres
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
    print("k = ", k, "\n", tabulate(data, headers=names, tablefmt="grid"), "\n\n")

k =  1 
 +-------------+-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |     spsolve |
+=============+=============+=============+=============+
| 0.000253076 | 0.000522412 | 0.000358433 | 0.000530711 |
+-------------+-------------+-------------+-------------+
| 0           | 0           | 0           | 0           |
+-------------+-------------+-------------+-------------+
| 0           | 0           | 0           | 0           |
+-------------+-------------+-------------+-------------+ 


k =  11 
 +-------------+-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |     spsolve |
+=============+=============+=============+=============+
| 0.00409857  | 0.0311927   | 0.00200459  | 0.00989809  |
+-------------+-------------+-------------+-------------+
| 8.29406e-11 | 9.94388e-11 | 8.29406e-11 | 1.85748e-15 |
+-------------+-------------+-------------+-------------+
| 1.13702e-11 | 4.19477e-10 | 1.13702e-11 | 5.3539

In [126]:
# Now we remove spsolve from the array of functions we want to test
# this way we can increase k a lot more and compare the last 3 solvers
arr = [cs111.CGsolve, cs111.Jsolve, scipy.sparse.linalg.cg]
names = arr
names = [item.__name__ for item in arr]
data = np.zeros((3,len(arr)))
for k in range(1,100,20):
    for i in range(len(arr)):
        name = arr[i].__name__
        A = cs111.make_A_3D(k)
        if (name == "LUsolve"):
            A = A.toarray()
        x_exact = np.ones(A.shape[0])
        b = A @ x_exact

        if (name == "CGsolve" or name == "Jsolve" or name == "cg"):
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b,tol=1e-10)
            data[0, i] = t.average
            x = arr[i](A,b,tol=1e-10)[0]
            data[1, i] = npla.norm(b-A@x)/npla.norm(b)
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
        else:
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b)
            data[0, i] = t.average
            if (name == "LUsolve"):
                x, relres = arr[i](A,b)
            else:
                x = arr[i](A,b)
                relres = npla.norm(b-A@x)/npla.norm(b)
            data[1, i] = relres
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
    print("k = ", k, "\n", tabulate(data, headers=names, tablefmt="grid"), "\n\n")

k =  1 
 +-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |
+=============+=============+=============+
| 0.000391526 | 0.000440237 | 0.000640517 |
+-------------+-------------+-------------+
| 0           | 0           | 0           |
+-------------+-------------+-------------+
| 0           | 0           | 0           |
+-------------+-------------+-------------+ 


k =  21 
 +-------------+-------------+-------------+
|     CGsolve |      Jsolve |          cg |
+=============+=============+=============+
| 0.0169864   | 0.185146    | 0.00875327  |
+-------------+-------------+-------------+
| 7.93236e-11 | 2.93861e-06 | 7.93235e-11 |
+-------------+-------------+-------------+
| 1.51347e-11 | 2.80628e-05 | 1.51347e-11 |
+-------------+-------------+-------------+ 


k =  41 
 +-------------+------------+-------------+
|     CGsolve |     Jsolve |          cg |
+=============+============+=============+
| 0.214162    | 1.23091    | 0.0903638   |
+

In [127]:
# remove jSolve and compare the cs111 cg to the scipy sparse cg
arr = [cs111.CGsolve, scipy.sparse.linalg.cg]
names = arr
names = [item.__name__ for item in arr]
data = np.zeros((3,len(arr)))
for k in range(50,150,25):
    for i in range(len(arr)):
        name = arr[i].__name__
        A = cs111.make_A_3D(k)
        if (name == "LUsolve"):
            A = A.toarray()
        x_exact = np.ones(A.shape[0])
        b = A @ x_exact

        if (name == "CGsolve" or name == "Jsolve" or name == "cg"):
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b,tol=1e-10)
            data[0, i] = t.average
            x = arr[i](A,b,tol=1e-10)[0]
            data[1, i] = npla.norm(b-A@x)/npla.norm(b)
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
        else:
            t = %timeit -n1 -r1 -q -p3 -o arr[i](A,b)
            data[0, i] = t.average
            if (name == "LUsolve"):
                x, relres = arr[i](A,b)
            else:
                x = arr[i](A,b)
                relres = npla.norm(b-A@x)/npla.norm(b)
            data[1, i] = relres
            data[2, i] = npla.norm(x_exact-x)/npla.norm(x_exact)
    print("k = ", k, "\n", tabulate(data, headers=names, tablefmt="grid"), "\n\n")

k =  50 
 +-------------+-------------+
|     CGsolve |          cg |
+=============+=============+
| 0.42884     | 0.23298     |
+-------------+-------------+
| 9.02992e-11 | 9.02991e-11 |
+-------------+-------------+
| 4.40113e-11 | 4.40113e-11 |
+-------------+-------------+ 


k =  75 
 +-------------+-------------+
|     CGsolve |          cg |
+=============+=============+
| 2.29602     | 1.31347     |
+-------------+-------------+
| 9.03741e-11 | 9.03741e-11 |
+-------------+-------------+
| 7.36996e-11 | 7.36996e-11 |
+-------------+-------------+ 


k =  100 
 +-------------+-------------+
|     CGsolve |          cg |
+=============+=============+
| 7.49157     | 4.00799     |
+-------------+-------------+
| 9.66652e-11 | 9.66651e-11 |
+-------------+-------------+
| 7.22236e-11 | 7.22236e-11 |
+-------------+-------------+ 


k =  125 
 +--------------+--------------+
|      CGsolve |           cg |
+==============+==============+
| 22.2413      | 11.1334      |
+----------

In [3]:
A = np.random.rand(8,5)
nrows, ncols = A.shape
print('shape of A:', (nrows, ncols))

U,sigma,Vt = spla.svd(A)

print('singular values:', sigma)
print('rank(A):', npla.matrix_rank(A))
print()

Ak = np.zeros(A.shape)
for k in range(len(sigma)):
    print('rank', npla.matrix_rank(Ak), 'approximation: 2-norm(A%d - A) =' % k, npla.norm(Ak-A,2))
    Ak += sigma[k] * np.outer(U[:,k], Vt[k,:])
print('rank', npla.matrix_rank(Ak), 'approximation: 2-norm(A%d - A) =' % len(sigma), npla.norm(Ak-A,2))

shape of A: (8, 5)
singular values: [3.2206 0.932  0.7762 0.5236 0.3938]
rank(A): 5

rank 0 approximation: 2-norm(A0 - A) = 3.2206391604613627
rank 1 approximation: 2-norm(A1 - A) = 0.9319898301507401
rank 2 approximation: 2-norm(A2 - A) = 0.7761525691538314
rank 3 approximation: 2-norm(A3 - A) = 0.5235878376487354
rank 4 approximation: 2-norm(A4 - A) = 0.3937765832244615
rank 5 approximation: 2-norm(A5 - A) = 1.814818392902269e-15
